# SQLalchemy

In [1]:
import sqlalchemy as db
import pandas as pd

user = 'root'
password = 'password'
server = 'localhost'
port = '3306'
database = 'test'

string_connection = 'mysql+pymysql://'+user+':'+password+'@'+server+':'+port+'/'+database

engine = db.create_engine(string_connection)
connection = engine.connect()
metadata = db.MetaData()

# Create
## Creating a Table

In [2]:
table_name = 'EMPLOYEES'

employees = db.Table(table_name, metadata,
              db.Column('Id', db.Integer(), primary_key=True),
              db.Column('name', db.String(50), nullable=False),
              db.Column('lastname', db.String(50)),
              db.Column('salary', db.Float(), default=100.0),
              db.Column('active', db.Boolean(), default=True)
              )

metadata.create_all(engine) 

## Inserting into database

In [3]:
#Inserting one record

query = db.insert(employees).values(Id=1, name='Alexander', lastname='Benavides', salary=60000.00, active=True) 
ResultProxy = connection.execute(query)

In [4]:
#Inserting Several records

emp1 = {'Id':'2',
        'name':'Juana', 
        'lastname':'lacubana', 
        'salary':35000, 
        'active':True
       }

emp2 = {'Id':'3', 
        'name':'Pedro', 
        'lastname':'Espitia', 
        'salary':40000, 
        'active':False
       }

query = db.insert(employees)

values_list = [emp1, emp2]
ResultProxy = connection.execute(query,values_list)

# Read

In [5]:
# Retrieve all records
results = connection.execute(db.select([employees])).fetchall()

# Create a Pandas Dataframe
df = pd.DataFrame(results)
if (len(results) > 0):
    df.columns = results[0].keys()
else:
    print('Empty Table')
df.head(4)


,Id,name,lastname,salary,active
0,1,Alexander,Benavides,60000.0,True
1,2,Juana,lacubana,35000.0,True
2,3,Pedro,Espitia,40000.0,False


In [6]:
# Reading table using SQL language
query = "SELECT * FROM EMPLOYEES E WHERE E.ID = 1"

cursor = connection.execute(query)
query_result = pd.read_sql(query,con=engine)
query_result

,Id,name,lastname,salary,active
0,1,Alexander,Benavides,60000.0,1


In [7]:
query = "SELECT * FROM EMPLOYEES"

cursor = connection.execute(query)

for row in cursor:
    print(row)

(1, 'Alexander', 'Benavides', 60000.0, 1)
(2, 'Juana', 'lacubana', 35000.0, 1)
(3, 'Pedro', 'Espitia', 40000.0, 0)


# Update

In [8]:
query = """
        UPDATE EMPLOYEES E
        SET NAME = 'New Name'
        WHERE E.ID = 1
        """

connection.execute(query)

query = "SELECT * FROM EMPLOYEES E WHERE E.ID = 1"

cursor = connection.execute(query)
query_result = pd.read_sql(query,con=engine)
query_result


,Id,name,lastname,salary,active
0,1,New Name,Benavides,60000.0,1


# Delete
## Truncate a Table

In [9]:
connection.execute("TRUNCATE TABLE EMPLOYEES")